In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
import shutil
import glob

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
ROOT_DIR = "/content/drive/MyDrive/Brain/Training"
number_of_images = {}

for dir in os.listdir(ROOT_DIR):
  number_of_images[dir] = len( os.listdir(os.path.join(ROOT_DIR, dir)) )

number_of_images.items()


dict_items([('no_tumor', 445), ('glioma_tumor', 891), ('pituitary_tumor', 959), ('meningioma_tumor', 981)])

In [ ]:
number_of_images.items()

dict_items([('no_tumor', 445), ('glioma_tumor', 891), ('pituitary_tumor', 959), ('meningioma_tumor', 981)])

In [ ]:
len(os.listdir("/content/drive/MyDrive/Brain/Training"))

4

In [ ]:
def dataFolder(p, split):

  if not os.path.exists("/content/drive/MyDrive/Brain/cancer" +p):
    os.mkdir("/content/drive/MyDrive/Brain/cancer"+ p)
    
    for dir in os.listdir(ROOT_DIR):
       os.makedirs("/content/drive/MyDrive/Brain/cancer"+p+"/"+dir)
       
       for img in np.random.choice(a = os.listdir(os.path.join(ROOT_DIR, dir)), 
                                   size = (math.floor(split*number_of_images[dir])-5),
                                           replace = False):
         o = os.path.join(ROOT_DIR,dir,img)
         D = os.path.join("/content/drive/MyDrive/Brain/cancer"+p, dir)
        #  os.remove(o)

  else:
    print(f"{p} Folder exsist ")


In [ ]:
dataFolder("Train", 0.7)

In [ ]:
dataFolder("val", 0.15)

In [ ]:
dataFolder("test", 0.15)

In [ ]:
#Model creation
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense, BatchNormalization, GlobalAvgPool2D,Activation
from keras.models import Sequential
from keras import Sequential
from tensorflow import keras
from tensorflow.keras import layers
import os

from keras.preprocessing.image import ImageDataGenerator

import keras

In [ ]:
#CNN model

model= Sequential()
model.add(Conv2D(filters= 16, kernel_size= (3,3), activation= 'relu', input_shape = (224,224,3) ))

model.add(Conv2D(filters= 32, kernel_size= (3,3), activation= 'relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters= 64, kernel_size= (3,3), activation= 'relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(filters= 128, kernel_size= (3,3), activation= 'relu'))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Dropout(rate=0.25))

model.add(Flatten())
model.add(Dense(units=64 , activation = 'relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(units=1 , activation = 'sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 16)      448       
                                                                 
 conv2d_1 (Conv2D)           (None, 220, 220, 32)      4640      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 110, 110, 32)     0         
 )                                                               
                                                                 
 conv2d_2 (Conv2D)           (None, 108, 108, 64)      18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 64)       0         
 2D)                                                             
                                                                 
 conv2d_3 (Conv2D)           (None, 52, 52, 128)       7

In [ ]:
model.compile(optimizer= 'adam', loss= keras.losses.binary_crossentropy, metrics=['accuracy'])

In [ ]:
# Data Generator
def preprocessingImage1(path):
  image_data = ImageDataGenerator(zoom_range=0.2, shear_range=0.2, rescale = 1/255, horizontal_flip= True)
  image = image_data.flow_from_directory(directory = path, target_size =(224,224), batch_size = 32, class_mode = 'binary')

  return image


In [ ]:
path="/content/drive/MyDrive/Brain/cancerTrain"
train_data = preprocessingImage1(path)

Found 0 images belonging to 4 classes.


In [ ]:
# Data Generator
def preprocessingImage2(path):
  image_data = ImageDataGenerator( rescale = 1/255)
  image = image_data.flow_from_directory(directory = path, target_size =(224,224), batch_size = 32, class_mode = 'binary')

  return image

In [ ]:
path="/content/drive/MyDrive/Brain/cancertest"
test_data = preprocessingImage1(path)

Found 0 images belonging to 4 classes.


In [ ]:
path="/content/drive/MyDrive/Brain/cancerval"
val_data = preprocessingImage1(path)

Found 0 images belonging to 4 classes.


In [ ]:
#early stopping and model check point
from keras.callbacks import EarlyStopping,ModelCheckpoint

#early stopping
es = EarlyStopping(monitor ="val_accuracy",min_delta= 0.01, patience= 3, verbose=1, mode = 'auto')

#model check point
mc = ModelCheckPoint(monitor ="val_accuracy",filepath = "./bestmodel.h5",verbose=1, save_best_only= True,  mode = 'auto')

cd = [es,mc]

NameError: ignored

In [ ]:
#model training
hs = model.fil_generator(generator= train_data, 
                         step_per_epoch= 8, 
                         epochs= 30 , 
                         verbose =1,
                         validation_data=val_data,
                         validation_steps=16,
                         callbacks= cd )

In [ ]:
#model graphical Interpretation

h= hs.history
h.keys()

dict_keys(['loss', 'accuracy', "val_loss", 'val_accuracy' ])

In [ ]:
plt.plot(h['accuracy'])
plt.plot(h['val_accuracy'], c= "red")

plt.title('acc vs val-acc')
plt.show()


In [ ]:
plt.plot(h['loss'])
plt.plot(h['val_loss'], c= "red")

plt.title('loss vs val-loss')
plt.show()

In [ ]:
#model accuracy

from keras import load_model

model = load_model("/content/bestmodel.h5")

In [ ]:
acc = model.evaluate_generator(test_data)[1]

print(f"accuracy of our model  is {acc*100} %")

In [ ]:
from keras.preprocessing.image import load_img, img_to_array

In [ ]:
path = ''

img = load_img(path, target_size =(224,224))
input_arr = img_to_array(img)/255

plt.imshow(input_arr)
plt.show()


input_arr.shape

input_arr = np.expand_dims(input_arr, axis=0)[0][0]
pred = model.predict_classes(input_arr)
pred

if pred ==0:
  print("the image is having  a Tumor")
else:
  print("the image is not having a tumor")

In [ ]:
import numpy as